In [1]:
import numpy as np
from skimage.registration import phase_cross_correlation
from skimage.transform import rescale
import imageio.v3 as iio
import matplotlib.pyplot as plt
from skimage.registration._phase_cross_correlation import _upsampled_dft
import pathlib
import sys
sys.path.insert(0, "../controller")
import tile_configuration
from util import get_image_dimensions
import shapely

In [2]:
prefix = pathlib.Path(r"C:\Users\davidek\microscope_ui\controller\photo\1732319758.459453")
tc = tile_configuration.TileConfiguration()                   
tc.load(prefix / "TileConfiguration.registered.registered.txt")                  
tc.move_to_origin()
s=tile_configuration.tile_config_to_shapely(prefix, tc) 
len(tc.images)

49

In [3]:
d = {}
for image1 in tc.images:
    for image2 in tc.images:
        if image1.filename == image2.filename:
            continue
        x1 = int(image1.x)
        y1 = int(image1.y)
        x2 = int(image2.x)
        y2 = int(image2.y)
        w1, h1 = get_image_dimensions(prefix / image1.filename)
        w2, h2 = get_image_dimensions(prefix / image2.filename)
        b1 = shapely.geometry.box(x1, y1, x1+w1, y1+h1)
        b2 = shapely.geometry.box(x2, y2, x2+w2, y2+h2)
        if b1.intersects(b2):
            d[image1.filename, image2.filename] = image1.x - image2.x, image1.y - image2.y
d

{(WindowsPath('test.1732319760.040808.jpg'),
  WindowsPath('test.1732319761.527621.jpg')): (-796.8612391222347,
  -4.396816200202764),
 (WindowsPath('test.1732319760.040808.jpg'),
  WindowsPath('test.1732319771.11828.jpg')): (-24.197918435960045,
  -255.75780263964072),
 (WindowsPath('test.1732319760.040808.jpg'),
  WindowsPath('test.1732319772.6083815.jpg')): (-818.8198354508645,
  -269.88330930549057),
 (WindowsPath('test.1732319760.040808.jpg'),
  WindowsPath('test.1732319782.2060425.jpg')): (-90.75377207858406,
  -758.4094577084923),
 (WindowsPath('test.1732319760.040808.jpg'),
  WindowsPath('test.1732319783.69891.jpg')): (-875.1440740977008,
  -781.3885006375889),
 (WindowsPath('test.1732319761.527621.jpg'),
  WindowsPath('test.1732319760.040808.jpg')): (796.8612391222347,
  4.396816200202764),
 (WindowsPath('test.1732319761.527621.jpg'),
  WindowsPath('test.1732319763.0175085.jpg')): (-851.1015904139286,
  10.398824966119506),
 (WindowsPath('test.1732319761.527621.jpg'),
  Window

In [26]:
d2 = {}
for image1 in tc.images:
    i1 = np.asarray(iio.imread(prefix / image1.filename).swapaxes(0, 1))
    print(i1.shape)
    for image2 in tc.images:
        if (image1.filename, image2.filename) in d:
            print("Consider", image1, image2)
            i2 = np.asarray(iio.imread(prefix / image2.filename).swapaxes(0, 1))

            shift, error, diffphase = phase_cross_correlation(i1, i2, upsample_factor=10, normalization=None)
            d2[image1.filename, image2.filename] = shift



(1280, 1024, 3)
Consider TileImage(filename=WindowsPath('test.1732319760.040808.jpg'), x=0.0, y=84.06942976389125) TileImage(filename=WindowsPath('test.1732319761.527621.jpg'), x=796.8612391222347, y=88.46624596409401)
Consider TileImage(filename=WindowsPath('test.1732319760.040808.jpg'), x=0.0, y=84.06942976389125) TileImage(filename=WindowsPath('test.1732319771.11828.jpg'), x=24.197918435960045, y=339.827232403532)
Consider TileImage(filename=WindowsPath('test.1732319760.040808.jpg'), x=0.0, y=84.06942976389125) TileImage(filename=WindowsPath('test.1732319772.6083815.jpg'), x=818.8198354508645, y=353.95273906938183)
Consider TileImage(filename=WindowsPath('test.1732319760.040808.jpg'), x=0.0, y=84.06942976389125) TileImage(filename=WindowsPath('test.1732319782.2060425.jpg'), x=90.75377207858406, y=842.4788874723835)
Consider TileImage(filename=WindowsPath('test.1732319760.040808.jpg'), x=0.0, y=84.06942976389125) TileImage(filename=WindowsPath('test.1732319783.69891.jpg'), x=875.1440

In [25]:
k=list(d)[0]
k2 = (k[1], k[0])
print(k2)
print(d[k2])
print(d2[k])
print(1280-d2[k2][0], d2[k2][1])

(WindowsPath('test.1732319761.527621.jpg'), WindowsPath('test.1732319760.040808.jpg'))
(796.8612391222347, 4.396816200202764)
[-516.3   27.4    0. ]
763.7 -27.4


In [ ]:
for k1 in d:
    print(d[k1], d2[k1])
    print()

In [ ]:
i1 = np.asarray(iio.imread(r"C:\Users\davidek\microscope_ui\controller\photo\1732319758.459453\test.1732319794.7590337.jpg"))
i2 = np.asarray(iio.imread(r"C:\Users\davidek\microscope_ui\controller\photo\1732319758.459453\test.1732319796.2479942.jpg"))


In [ ]:
image = i2.swapaxes(0,1)
offset_image = i1.swapaxes(0,1)

In [ ]:
#test.1732319794.7590337.jpg; ; (874.8389520097386, 1561.7998672387732)
#test.1732319796.2479942.jpg; ; (1731.0772784713163, 1548.905338786314)


In [ ]:

# pixel precision first
shift, error, diffphase = phase_cross_correlation(image, offset_image, upsample_factor=10, normalization=None)
print(shift)
print(1280-shift[0], shift[1])



In [ ]:
def get_shift(i1, i2):
    # compute the cross-power spectrum of the two images:
    image_product = np.fft.fft2(i1) * np.fft.fft2(i2).conj()

    # compute the (not-normalized) cross-correlation between 
    # the two images:
    cc_image = np.fft.ifft2(image_product)

    # for visualization reasons, shift the zero-frequency 
    # component to the center of the spectrum:
    #cc_image_fftshift = np.fft.fftshift(cc_image)
    shape = i1.shape
    # find the peak in cc_image: 
    maxima = np.unravel_index(np.argmax(np.abs(cc_image)), shape)
    midpoints = np.array([np.fix(axis_size / 2) for axis_size in shape])
    float_dtype = image_product.real.dtype
    shifts = np.stack(maxima).astype(float_dtype, copy=False)
    shifts[shifts > midpoints] -= np.array(shape)[shifts > midpoints]
    #print(f"detected shifts: {shifts[1], shifts[0]}")
    return shifts


In [ ]:

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 4, 1)
ax2 = plt.subplot(1, 4, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 4, 3)
ax4 = plt.subplot(1, 4, 4)

ax1.imshow(image, cmap='gray')
ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(offset_image.real, cmap='gray')
ax2.set_axis_off()
ax2.set_title('Offset image')

# Show the output of a cross-correlation to show what the algorithm is
# doing behind the scenes
image_product = np.fft.fft2(image) * np.fft.fft2(offset_image).conj()
cc_image = np.fft.fftshift(np.fft.ifft2(image_product))

ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Cross-correlation")


# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = _upsampled_dft(image_product, 150, 100, (shift * 100) + 75).conj()
ax4.imshow(cc_image.real)
ax4.set_axis_off()
ax4.set_title("Supersampled XC sub-area")
plt.show()

